In [1]:
!pip install nvidia-modulus nvidia-modulus-sym

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.1/772.1 kB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.0/292.0 kB 20.5 MB/s eta 0:00:00
  Installing build dependencies ... don

In [1]:
import os
import warnings
import torch # (important for visualization part of the tutorial)
import matplotlib.pyplot as plt
import sympy
from sympy import Symbol, Eq, Abs


In [12]:
from modulus.sym.eq.pde import PDE
from sympy import symbols, diff, Matrix

class Maxwell(PDE):
    def __init__(self):
        # Define spatial and time variables
        x, y, z, t = symbols('x y z t')

        # Define components of electric field (E) and magnetic field (B)
        E_x, E_y, E_z = symbols('E_x E_y E_z')
        B_x, B_y, B_z = symbols('B_x B_y B_z')

        # Define charge density (rho) and current density (J)
        rho = symbols('rho')
        J_x, J_y, J_z = symbols('J_x J_y J_z')

        # Define permittivity (epsilon_0) and permeability (mu_0)
        epsilon_0, mu_0 = symbols('epsilon_0 mu_0')

        # Electric field vector (E) and magnetic field vector (B)
        E = Matrix([E_x, E_y, E_z])
        B = Matrix([B_x, B_y, B_z])
        J = Matrix([J_x, J_y, J_z])

        # Define a helper function for computing the curl of a vector field
        def curl(F, x, y, z):
            F_x, F_y, F_z = F
            return Matrix([
                diff(F_z, y) - diff(F_y, z),  # curl component in x-direction
                diff(F_x, z) - diff(F_z, x),  # curl component in y-direction
                diff(F_y, x) - diff(F_x, y)   # curl component in z-direction
            ])

        # Gauss's law for electric field: ∇·E = ρ / ε_0
        gauss_electric = diff(E_x, x) + diff(E_y, y) + diff(E_z, z) - rho / epsilon_0

        # Gauss's law for magnetism: ∇·B = 0
        gauss_magnetic = diff(B_x, x) + diff(B_y, y) + diff(B_z, z)

        # Faraday's law of induction: ∇×E = -∂B/∂t
        faraday_law = curl(E, x, y, z) + diff(B, t)

        # Ampère's law (with Maxwell's correction): ∇×B = μ_0 J + μ_0 ε_0 ∂E/∂t
        amperes_law = curl(B, x, y, z) - mu_0 * J - mu_0 * epsilon_0 * diff(E, t)

        # Store the equations in self.equations
        self.equations = {
            'Gauss_Electric': gauss_electric,
            'Gauss_Magnetic': gauss_magnetic,
            'Faraday_Law': faraday_law,
            'Amperes_Law': amperes_law
        }

In [13]:
from modulus.sym.models.fully_connected import FullyConnectedArch
from modulus.sym.models.activation import Activation
from modulus.sym.key import Key

flow_net = FullyConnectedArch(
    # Science & Engineering Knowledge
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u"), Key("v"), Key("p")],
    # Neural Networks Knwoledge (You don't need to know...)
    layer_size=512,
    nr_layers=6,
    skip_connections=False,
    activation_fn=Activation.SILU,
    adaptive_activations=False,
    weight_norm=True
)

In [14]:
flow_net

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=2, out_features=512, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=512, out_features=512, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=512, out_features=3, bias=True)
    )
  )
)

In [16]:
# Create an instance of Maxwell class
maxwell = Maxwell()

# Create nodes from Maxwell equations and other networks
nodes = maxwell.make_nodes() + [flow_net.make_node(name="flow_network")]


In [17]:
nodes

In [ ]:
from modulus.sym.geometry.primitives_2d import Rectangle

